# PySpark

### Connecting to MngoDB using PySpark

In [1]:
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

my_spark = SparkSession \
    .builder \
    .appName("hadoopapp") \
    .config("spark.sql.broadcastTimeout", "36000") \
    .getOrCreate()

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [8]:
df = spark.read.csv('hdfs://localhost:9000/user/hadoop/data/data.csv')

In [9]:
df1 = spark.read.csv('hdfs://localhost:9000/user/hadoop/data/test_data.csv')

In [38]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField

schema = StructType([StructField("logging_level", StringType(), True),StructField("timestamp", StringType(), True),StructField("downloader_id", StringType(), True),StructField("retrieval_stage", StringType(), True),StructField("description", StringType(), True)])

NameError: name 'StringType' is not defined

In [30]:
schema = new StructType()
      .add("RecordNumber",IntegerType,true)
      .add("Zipcode",IntegerType,true)
      .add("ZipCodeType",StringType,true)
      .add("City",StringType,true)
      .add("State",StringType,true)
df2  = spark.read.format('csv').schema(schema).load('hdfs://localhost:9000/user/hadoop/data/test_data.csv')

SyntaxError: invalid syntax (<ipython-input-30-b2dc0179ef7b>, line 1)

In [10]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



In [12]:
df1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



_c0 - logging_level

_c1 - timestamp

_c2 - downloader_id

_c3 - retrieval_stage

_c4 - description

In [13]:
df.registerTempTable("Log")

In [14]:
df1.registerTempTable("Log1")

### 2. How many records does the table contain?

In [15]:
sqlContext =SQLContext(sc)
result_data = sqlContext.sql('select count(_c0) from Log')
result_data.show()

+----------+
|count(_c0)|
+----------+
|   9669731|
+----------+



### 3. Count the number of WARNing messages.

In [16]:
sqlContext =SQLContext(sc)
result_data = sqlContext.sql('select count(_c2),_c0 from Log where _c0 like \"WARN\" group by _c0')
result_data.show()

+----------+----+
|count(_c2)| _c0|
+----------+----+
|    132158|WARN|
+----------+----+



### 4.  How many repositories were processed in total? Use the api_client.

In [17]:
sqlContext =SQLContext(sc)
result_data = sqlContext.sql(' select count(*) from (select distinct (regexp_extract(_c4,\"api.github.com/[a-zA-Z0-9-]+/[a-zA-Z0-9-]+/\",0)) as  url from Log1) as temp')
result_data.show()

+--------+
|count(1)|
+--------+
|       7|
+--------+



### 5.   Which 10 clients did the highest HTTP requests?

In [7]:
sqlContext =SQLContext(sc)
result_data = sqlContext.sql(" select _c2 , count(*) from Log1 where _c4 like \'%http:%\' or _c4 like \'%https:%\' group by _c2 ORDER BY COUNT(*) DESC limit 1 ")
result_data.show()

IllegalArgumentException: 'Unsupported class file major version 55'

### 6.   Which 10 client did the highest FAILED HTTP requests?

In [9]:
sqlContext = SQLContext(sc)
result_data = sqlContext.sql(" select _c2,count(*) from Log where _c4 like \'%http:%\' or _c4 like \'%https:%\' and _c4 like \'Failed%\' group by _c2 ORDER BY COUNT(*) DESC limit 10 ")
result_data.show()

IllegalArgumentException: 'Unsupported class file major version 55'

### 7.   What is the most active hour of day?

In [ ]:
sqlContext =SQLContext(sc)
result_data = sqlContext.sql(" select count(*),temp.time from (select extract(hour from timestamp ) as time from Log) as temp group by temp.time order by count(*) DESC limit 1 ")
result_data.show()

### 8. What is the most active repository?

In [56]:
sqlContext =SQLContext(sc)
result_data = sqlContext.sql("select count(*),substring(cast(temp.url as STRING),22) from (select (regexp_extract(_c4,\"api.github.com/repos/[a-zA-Z0-9-]+/[a-zA-Z0-9-]+/\",0)) as  url from Log) as temp where char_length(temp.url) > 0 group by  temp.url order by count(*) DESC limit 1")
result_data.show()

IllegalArgumentException: 'Unsupported class file major version 55'

### 9. Which access keys are failing most often?

In [ ]:
sqlContext =SQLContext(sc)
result_data = sqlContext.sql("select count(*),substring(cast(temp.access as STRING),9,11) from (select (regexp_extract(description,\'Access: [0-9a-zA-Z]+,\',0)) as  access from Log where description like \'Failed%\') as temp group by temp.access order by count(*) DESC limit 10")
result_data.show()

### 10. Compute the number of different repositories accessed by the client ghtorrent-22 . Note the time taken by your query.

#### With Index

In [ ]:
import time
current_time = time.time()
sqlContext =SQLContext(sc)
result_data = sqlContext.sql("select count(temp.url) from (select downloader_id,(regexp_extract(description,\'api.github.com/[a-zA-Z0-9-]+/[a-zA-Z0-9-]+/\',0)) as  url from Log) as temp where temp.downloader_id = 22")
result_data.show()
print(time.time()-current_time)

### 11. Now drop your index and compute the number of different repositories accessed by the client ghtorrent-22. Note the time taken by query now.

#### without index

In [ ]:
import time
current_time = time.time()
sqlContext =SQLContext(sc)
result_data = sqlContext.sql("select count(temp.url) from (select downloader_id,(regexp_extract(description,\'api.github.com/[a-zA-Z0-9-]+/[a-zA-Z0-9-]+/\',0)) as  url from Log) as temp where temp.downloader_id = 22")
result_data.show()
print(time.time()-current_time)

#### The time taken to execute the query when index was created is less than the time taken when index is not there.

Time1 = 

Time2 =

### 12. Read in the CSV file into another table call it interesting. How many records are there?

mongoimport --db bda --collection interesting --type csv --fields id,timestamp,URL,OwnerId,Name,Language,created_at,forked_from,deleted,updated_at --file important-repos.csv

In [ ]:
df2 = spark.read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("database","bda")\
    .option("collection", "interesting")\
    .load()

In [ ]:
df2.printSchema()

In [ ]:
df2.registerTempTable("interesting")

In [ ]:
sqlContext =SQLContext(sc)
result_data = sqlContext.sql(" select count(*) from interesting ")
result_data.show()

### 13. How many records in the log file refer to entries in the interesting file?

In [ ]:
sqlContext =SQLContext(sc)
result_data = sqlContext.sql("select count(newfile._id) from (select temp._id,cast(split(substring(cast(temp.url as String),22),'/}')[0] as String ) as account from (select _id,(regexp_extract(description,\'api.github.com/repos/[a-zA-Z0-9\-.]+/[a-zA-Z0-9\-.]+/\',0)) as  url from Log) as temp where char_length(temp.url) > 0) as newfile where EXISTS (select  id from interesting where interesting.URL like \'%\' || newfile.account || \'%\' )")
result_data.show()

### 14. How many records in the log file refer to entries in the interesting file?

In [14]:
!java -version

openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


In [ ]:

"select count(*),substring(cast(temp.url as text),22) from (select (regexp_extract(description,\'api.github.com/repos/[a-zA-Z0-9-]+/[a-zA-Z0-9-]+/\',0)) as  url from Log) as temp where size(temp.url) > 0 group by  temp.url order by count(*) DESC limit 1"

export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

In [ ]:
print(' select count(*),substring(cast(temp.url as text),22) from (select (REGEXP_EXTRACT(description,'api.github.com/repos/[a-zA-Z0-9-]+/[a-zA-Z0-9-]+/')) as url from Log) as temp where array_length(temp.url,1) > 0 group by temp.url order by count(*) DESC limit 1')